<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/Andrey/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [6]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
100% 35.7M/35.7M [00:00<00:00, 136MB/s]
Using competition: 26jun23
  teamId  teamName                   submissionDate       score    
--------  -------------------------  -------------------  -------  
10612701  Никита Угрюмов             2023-06-28 23:59:12  0.87976  
10612695  Ne1tr0N [Maxim Mashtaler]  2023-06-28 22:04:00  0.87931  
10612706  Michael Kondrashov         2023-06-28 14:52:04  0.87746  
10616508  Ulyana Klushova            2023-06-28 20:38:57  0.87485  
10612698  Alexandr Boldirev          2023-06-28 21:14:30  0.87344  
10612715  Stepan Vysotsky            2023-06-28 17:57:40  0.87076  
10614521  Konstantin Zeigman         2023-06-28 13:40:11  0.86889  
10614699  Dmitriy Geyzha             2023-06-28 12:50:43  0.86653  
10613713  Mikhail Vovk               2023-06-28 15:19:07  0.86403  
10612680  esheshka                   2023-06-29 05:05:39  0.85417  
10618717  TimoteoByalko

In [8]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 10.9 s, sys: 1.3 s, total: 12.2 s
Wall time: 1min 33s


In [702]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier as SC
from sklearn.neighbors import KNeighborsClassifier as KNC

from sklearn.model_selection import GridSearchCV as GSCV
from sklearn.model_selection import RandomizedSearchCV as RSCV


In [10]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc')

CPU times: user 7.03 ms, sys: 1.14 ms, total: 8.17 ms
Wall time: 24.9 ms


In [11]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc');

CPU times: user 4.94 ms, sys: 0 ns, total: 4.94 ms
Wall time: 5.03 ms


In [12]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid'); gender.T  # load gender labels. you need to predict the missing labels
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')  # split into Train and test (=Validation) targets

CPU times: user 6.97 ms, sys: 1.83 ms, total: 8.8 ms
Wall time: 9.79 ms


cid,0,1,2,3,4,...,8395,8396,8397,8398,8399
gender,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0


In [13]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm']); dfTrx  # load bank card's transactions. Ensure ordered CID

CPU times: user 8.74 s, sys: 1.52 s, total: 10.3 s
Wall time: 18.2 s


,cid,dt,tm,amt,mcc,ttc,tid
0,0,1,00:00:00,-1305,56,4,NaN
1,0,1,00:00:00,-4821,56,4,NaN
2,0,2,22:48:17,-56156,22,37,NaN
3,0,3,00:00:00,-3708,56,4,NaN
...,...,...,...,...,...,...,...
3749574,8399,450,00:00:00,-43831,56,4,PS026577
3749575,8399,456,17:21:07,89833,123,128,888917
3749576,8399,456,18:59:26,247062,124,108,738051
3749577,8399,456,19:01:21,-336874,22,25,738051


In [14]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


In [15]:
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 10)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 120 sec. Good luck!


# **Тесты**

In [800]:
df_tid, df_no_tid = dfTrx[~dfTrx.tid.isna()], dfTrx[dfTrx.tid.isna()]

In [806]:
df_tid[df_tid.tid == 'PS026577']

,cid,dt,tm,amt,mcc,ttc,tid
664304,1525,303,20,-398132,50,4,PS026577
3749574,8399,450,0,-43831,56,4,PS026577


# **My Code**

In [749]:
#@title Настройки

large_count = 2000 * 0.85
large_sum = -1000000000 * 0.85

In [750]:
#@title Определение наиболее используемых mcc
%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > large_count].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < large_sum].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 1.08 s, sys: 204 ms, total: 1.28 s
Wall time: 1.33 s


35

In [751]:
#@title Определение наиболее используемых ttc
%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > large_count].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < large_sum) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 976 ms, sys: 201 ms, total: 1.18 s
Wall time: 1.21 s


25

# **Начало создания df по cid**

In [788]:
#@title Сумма

def s(to_df):
    return to_df / to_df.sum(axis=0)
    # return to_df / np.sqrt((to_df**2).sum(axis=0))
    # return to_df / (np.sqrt(to_df**2)).sum(axis=0)

In [789]:
#@title Настройки

stat = ['count', 'sum', 'median']
is_PCA = 0

In [790]:
%%time
df = df_.copy(deep=True)

CPU times: user 224 µs, sys: 3 µs, total: 227 µs
Wall time: 232 µs


In [791]:
#@title MCC
%%time
to_df = df_.copy(deep=True)

for mcc_index in most_popular_mcc:
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

stat_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_mcc.columns += '_mcc'
to_df = to_df.join(stat_by_other_mcc, how='left')

to_df = s(to_df) * 75
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.81 s, sys: 3.98 ms, total: 1.82 s
Wall time: 1.91 s


In [792]:
#@title TTC
# %%time
# to_df = df_.copy(deep=True)

# for ttc_index in most_popular_ttc:
#     sum_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
#     to_df = to_df.join(sum_amt_by_cid_and_ttc.amt, on = 'cid', how='left').rename(columns = {'amt': str(ttc_index) + '_ttc'})

# sum_amt_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc)].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
# to_df = to_df.join(sum_amt_by_other_ttc.amt, on = 'cid', how='left').rename(columns = {'amt': 'other_ttc'})

# if is_PCA == 2:
#     to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1]), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
#     to_df.columns += df.shape[1]
# df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for ttc_index in most_popular_ttc:
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

stat_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_ttc.columns += '_ttc'
to_df = to_df.join(stat_by_other_ttc, how='left')

to_df = s(to_df) * 1/2
df = df.join(to_df, on = 'cid', how='left')
# to_df

In [793]:
#@title AMT

to_df = df_.copy(deep=True)

to_df = dfTrx[['cid', 'amt']].groupby('cid').agg(stat).add_prefix('').droplevel(0, axis='columns')
to_df = df_.join(to_df, how='left').fillna(0)

to_df = s(to_df) * 2/3
df = df.join(to_df, on = 'cid', how='left')

In [794]:
#@title Time
%%time
to_df = df_.copy(deep=True)

for hour in range(9):
#     sum_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
#     df = df.join(sum_amt_by_cid_and_hour.amt, on = 'cid', how='left').rename(columns = {'amt': str(hour) + '_tm'})

    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(9, 24):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 2), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.47 s, sys: 14.9 ms, total: 1.48 s
Wall time: 1.5 s


In [795]:
#@title Date
%%time
to_df = df_.copy(deep=True)

# for weekday in range(7):
#     sum_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
#     to_df = to_df.join(sum_amt_by_cid_and_weekday.amt, on = 'cid', how='left').rename(columns = {'amt': str(weekday) + '_wd'})

for weekday in [0, 3, 4, 5, 6]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')
# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 2), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]
# to_df = pd.DataFrame(to_df.sum(axis=1))
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for weekday in [1, 2]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')


# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 3), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.1 s, sys: 7.97 ms, total: 1.11 s
Wall time: 1.12 s


In [796]:
df = df.fillna(0).astype('float')
df.columns = [str(c) for c in df.columns]

# **Тест**

# **Обучение**

In [797]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.3, random_state=0) # split train observation into local train/test sets (to score performance)

# estimators = [
#     ('rfc', RFC(n_estimators=10, random_state=42)),
#     # ('knc', KNC()),
#     # ('svr', make_pipeline(StandardScaler(),
#     ('lr', LR(random_state=0, n_jobs=-1, max_iter=1000)),
#     # ('svc', SVC(probability=True)),
#     # ('gbc0', GBC()),
#     ('gbc1', GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2))
# ]
# model = SC(estimators=estimators, final_estimator=GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2))

# model = LR(random_state=0, n_jobs=-1, max_iter=1000)                             # always seed your experiments for reproducibility
# model = RFC()
# model = KNC()
# params = {
#     'n_neighbors': np.arange(1, 11),
#     'metric': ['manhattan', 'euclidean'],
#     'weights': ['uniform', 'distance']
# }
# model = DTC()
# model = SVC(probability=True)
# model = GBC()
model = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
# params = {
#     'max_depth': [None, 2, 3, 4, 5],
# }

# model = GSCV(model, params, return_train_score=True, cv=5, scoring='accuracy', n_jobs=-1)
pY = model.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
# print(model.best_params_)

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))       # fit the model and show test error
print(pY.sum() / pY.shape[0])

Local AUC ROC 0.85706569664903
0.43491221906428
CPU times: user 25.9 s, sys: 38.2 ms, total: 26 s
Wall time: 27.2 s


In [276]:
tmr.ShowTime()

Runtime is 1717 sec > 120 sec limit!!!


In [735]:
# model = LR(random_state=0, n_jobs=-1, max_iter=1000).fit(trainX, trainY.gender)  # now train again, on the full training set
# model = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model = SC(estimators=estimators)
model.fit(trainX, trainY.gender)

StackingClassifier(estimators=[('knc', KNeighborsClassifier()),
                               ('gbc1',
                                GradientBoostingClassifier(max_depth=2,
                                                           n_estimators=150))])

In [736]:
pY = pd.DataFrame(model.predict_proba(testX)[:,1], columns=['gender'])
pY.index.name = 'cid'

print(pY.sum() / pY.shape[0])

gender    0.443865
dtype: float64


In [737]:
pY.reset_index().to_csv('💳Baseline.csv', index=False, float_format='%10f')